In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import sys
import os
import scipy
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

In [ ]:
%cd /content/sample_data
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
!gzip -d GoogleNews-vectors-negative300.bin.gz

/content/sample_data
--2021-09-08 12:28:03--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.107.22
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.107.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  42.8MB/s    in 38s     

2021-09-08 12:28:42 (41.1 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
test_data=pd.read_parquet('/content/drive/MyDrive/recipe_kg/recipes.parquet')
print(test_data.shape, test_data.head())
col=list(test_data.columns)
print(col)
test_data.head()

(522517, 28)    RecipeId  ...                                 RecipeInstructions
0      38.0  ...  [Toss 2 cups berries with sugar., Let stand fo...
1      39.0  ...  [Soak saffron in warm milk for 5 minutes and p...
2      40.0  ...  [Into a 1 quart Jar with tight fitting lid, pu...
3      41.0  ...  [Drain the tofu, carefully squeezing out exces...
4      42.0  ...  [Mix everything together and bring to a boil.,...

[5 rows x 28 columns]
['RecipeId', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime', 'TotalTime', 'DatePublished', 'Description', 'Images', 'RecipeCategory', 'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts', 'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent', 'SaturatedFatContent', 'CholesterolContent', 'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent', 'RecipeServings', 'RecipeYield', 'RecipeInstructions']


,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38.0,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00+00:00,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,Frozen Desserts,"[Dessert, Low Protein, Low Cholesterol, Health...","[4, 1⁄4, 1, 1]","[blueberries, granulated sugar, vanilla yogurt...",4.5,4.0,170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,None,"[Toss 2 cups berries with sugar., Let stand fo..."
1,39.0,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29 13:12:00+00:00,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,Chicken Breast,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...","[1, 4, 2, 2, 8, 1⁄4, 8, 1⁄2, 1, 1, 1⁄4, 1⁄4, 1...","[saffron, milk, hot green chili peppers, onion...",3.0,1.0,1110.7,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,None,[Soak saffron in warm milk for 5 minutes and p...
2,40.0,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05 19:52:00+00:00,This is from one of my first Good House Keepi...,[https://img.sndimg.com/food/image/upload/w_55...,Beverages,"[Low Protein, Low Cholesterol, Healthy, Summer...","[1 1⁄2, 1, None, 1 1⁄2, None, 3⁄4]","[sugar, lemons, rind of, lemon, zest of, fresh...",4.5,10.0,311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,None,"[Into a 1 quart Jar with tight fitting lid, pu..."
3,41.0,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03 14:54:00+00:00,This dish is best prepared a day in advance to...,[https://img.sndimg.com/food/image/upload/w_55...,Soy/Tofu,"[Beans, Vegetable, Low Cholesterol, Weeknight,...","[12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1⁄2, 1⁄...","[extra firm tofu, eggplant, zucchini, mushroom...",4.5,2.0,536.1,24.0,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"[Drain the tofu, carefully squeezing out exces..."
4,42.0,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19 06:19:00+00:00,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,Vegetable,"[Low Protein, Vegan, Low Cholesterol, Healthy,...","[46, 4, 1, 2, 1]","[plain tomato juice, cabbage, onion, carrots, ...",4.5,11.0,103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,None,"[Mix everything together and bring to a boil.,..."


In [ ]:
import nltk
from nltk.stem import PorterStemmer
import random

porter = PorterStemmer()

ingre_set = set()
ingre_sen = []
def append_ingre_sen(x):
  sen_orig = list(x)
  sentences = []
  if sen_orig == [None]:
    return x
  for w in sen_orig:
    ingre_set.add(w.lower().replace('/',' ').replace('.','').replace('(','').replace(')',''))
    w = [porter.stem(i) for i in w.lower().replace('/',' ').replace('.','').replace('(','').replace(')','').split(' ')]
    sentences.extend(w)
  ingre_sen.append(sentences)
  for i in range(2):
    random.shuffle(sentences)
    ingre_sen.append(sentences)
  return x
test_data['RecipeIngredientParts'] = test_data['RecipeIngredientParts'].apply(append_ingre_sen)

In [ ]:
# from gensim.models import KeyedVectors
# model_google = KeyedVectors.load_word2vec_format("/content/sample_data/GoogleNews-vectors-negative300.bin",binary = True)

In [ ]:
#option one:
# from gensim.models import Word2Vec

# model = Word2Vec( size=300, window=5, min_count=1, workers=4)
# model.build_vocab(ingre_sen)
# model.intersect_word2vec_format('/content/sample_data/GoogleNews-vectors-negative300.bin', lockf=1.0, binary=True)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
vocab_dict = {k:model.wv[k] for k, v in model.wv.vocab.items()}
main_label = ['Apple',
'Banana',
'Bell pepper',
'Broccoli',
'Cabbage',
'Carrot',
'Cheese',
'Coconut',
'Cucumber',
'Egg',
'Fish',
'Grapes',
'Lemon',
'Mango',
'Milk',
'Mushroom',
'Oranges',
'Peach',
'Pear',
'Pineapple',
'Potatoes',
'Pumpkin',
'Seafood',
'Shrimp',
'Strawberry',
'Tomatoes',
'Watermelon',
'Winter melon',
'Garlic',
'Corn',
'Eggplant',
'Lettuce',
'Onion',
'Scallion',
'chicken',
'beef',
'lamb',
'pork',
'sauce',
'duck',
'meatball',
'wine',
'berries',
'crabmeat',
'kiwi',
'bitter melon',
'pepper',
'peas',
'ginger',
'shells',
'chili',
'ham',
'sausage',
'butter',
'bread',
'rice',
'vanilla']
main2sub = {}
miss_ingre = []
for i in main_label:
  main2sub[i] = []
for i,e in enumerate(ingre_set):
  closest_m = None
  closest_sim = 0
  # if i > 1000:
  #   break
  # elif i < 500:
  #   continue
  for m in main_label:

    # option one:
    # e_vec = np.sum([vocab_dict[porter.stem(i)] for i in e.lower().split(' ')],axis=0)
    # m_vec = np.sum([vocab_dict[porter.stem(i)] for i in m.lower().split(' ')],axis=0)
    # e_similarity = cosine_similarity([e_vec],[m_vec])[0][0]

    # option two:
    # e_vecs = [vocab_dict[porter.stem(i)] for i in e.lower().split(' ')]
    # m_vec = [vocab_dict[porter.stem(i)] for i in m.lower().split(' ')]
    # e_similarity = np.mean(cosine_similarity(e_vecs,m_vec))

    # option three:
    e_vec = np.dot(np.arange(1,1+len(e.lower().split(' '))*0.5,0.5),np.array([vocab_dict[porter.stem(i)] for i in e.lower().split(' ')]))
    m_vec = np.dot(np.arange(1,1+len(m.lower().split(' '))*0.5,0.5),np.array([vocab_dict[porter.stem(i)] for i in m.lower().split(' ')]))
    e_similarity = cosine_similarity([e_vec],[m_vec])[0][0]

    # option four:
    # e_vecs = [vocab_dict[porter.stem(i)] for i in e.lower().split(' ')]
    # m_vec = [vocab_dict[porter.stem(i)] for i in m.lower().split(' ')]
    # e_similarity = np.max(cosine_similarity(e_vecs,m_vec))

    if e_similarity > closest_sim:
      closest_sim = e_similarity
      closest_m = m
  # print(closest_m,'|',e,'|',closest_sim)

  if closest_sim > 0.5:
      main2sub[closest_m].append(e)
      # print(closest_m,'|',e,'|',closest_sim)

  else:
      miss_ingre.append(e)
      # print(closest_m,'|',e,'|',closest_sim)
print(main2sub)
print(miss_ingre)
import pickle as pk
with open('/content/drive/MyDrive/IRSProjectSharedFolder/align_name/main2sub.pickle', 'wb') as handle:
    pk.dump(main2sub, handle)
with open('/content/drive/MyDrive/IRSProjectSharedFolder/align_name/miss_ingre.pickle', 'wb') as handle:
    pk.dump(miss_ingre, handle)

{'Apple': ['australian sweet apple chutney', 'apple ring', 'tart baking apple', 'braeburn apples', 'braeburn apple', 'sweet apple cider', 'apples', 'green apples', 'dry apple cider', 'tart baking apples', 'brown apple vinegar', 'fuji apples', 'winesap apple', 'yellow apples', 'cracker barrel fried apples', 'dutch apples', 'rome apples', 'apple', 'yellow apple', 'green apple', 'fuji apple', 'tart dessert apple', 'stayman apples', 'pie apples', 'sliced apples', 'apple butter', 'apple brandy', 'crisp red apples', 'tart green apple', 'cooking apples', 'heinz apple cider vinegar', 'pippin apple', "drunken apples - like famous dave's", 'sparkling apple cider', 'jonathan apples', 'mcintosh apples', 'cranberry sauce with apple cider', 'sweet apple', 'hard apple cider', 'red delicious apple', 'sour apples', 'tart cooking apples', 'seedless raisins', 'spiced apple cider', 'white seedless raisin', 'pie apple', 'green apple vodka', 'rome apple', 'red apple', 'red baking apple', 'tart green apples'

In [ ]:
for i in np.arange(1,1.5+3*0.5,0.5):
  print(i)

1.0
1.5
2.0
2.5


In [ ]:
# model.train(ingre_sen,total_examples=len(ingre_sen),epochs=10)

(4190008, 7065667)

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(ingre_sen, size=100, window=5, min_count=1, workers=4)

In [ ]:
sims = model.wv.most_similar(porter.stem('steak'), topn=50)
print(porter.stem('steak'),sims)
print(model.wv.similarity(porter.stem('steak'), porter.stem('beef')))

steak [('sirloin', 0.7395126819610596), ('filet', 0.7349370121955872), ('tenderloin', 0.7280069589614868), ('burger', 0.6725448369979858), ('meatloaf', 0.6439660787582397), ('brisket', 0.6382268667221069), ('reuben', 0.6013650894165039), ('sandwich', 0.5976066589355469), ('bacon', 0.5947791337966919), ('cutlet', 0.5928877592086792), ('bresaola', 0.5922760963439941), ('fillet', 0.5907865762710571), ('scallopini', 0.5902199745178223), ('coleslaw', 0.5816993713378906), ('salad', 0.5794450044631958), ('sushi', 0.5772001147270203), ('veal', 0.5758250951766968), ('meat', 0.5724613666534424), ('rigatoni', 0.5680099725723267), ('queso', 0.5676813125610352), ('teriyaki', 0.5632550120353699), ('risotto', 0.5575823783874512), ('prosciutto', 0.5575358867645264), ('lasagna', 0.5564424991607666), ('chimichurri', 0.5564325451850891), ('mortadella', 0.5534413456916809), ('polenta', 0.5527387857437134), ('spatchcock', 0.5507605075836182), ('panang', 0.5499935150146484), ('taquito', 0.5495648980140686),

In [ ]:
import pickle as pk

main2sub = pk.load(open('/content/drive/MyDrive/IRSProjectSharedFolder/align_name/main2sub.pickle', 'rb'))

In [ ]:
for main in main2sub:
  print(main+'\t'+','.join(main2sub[main][:3]))

Apple	australian sweet apple chutney,apple ring,tart baking apple
Banana	banana,banana chips,low-fat banana instant pudding mix
Bell pepper	red bell peppers,frozen broccoli with red pepper,frozen bell peppers and onions
Broccoli	broccoli sprouts,broccoli rabe,broccoli stem
Cabbage	savoy cabbage,hungarian sweet 'n' sour cabbage soup,celery root
Carrot	quick-cooking pearl barley,raw carrots,carrots
Cheese	sargento chefstyle cheddar cheese,fat-free american cheese slices,fat-free cheddar cheese
Coconut	fresh coconut,canned unsweetened coconut milk,low-fat coconut milk
Cucumber	cucumber tzatziki,english seedless cucumber,jicama
Egg	jumbo egg yolks,egg custard,extra-large egg
Fish	fish curry powder,bamboo shoots,dried kaffir lime leaves
Grapes	champagne grapes,fresh red cherries,red cherries
Lemon	lemon slice,candied lemon peel,fresh lemon rind
Mango	papaya nectar,mango peach chutney,mango puree
Milk	2% low-fat chocolate milk,half and half milk,regular margarine
Mushroom	mushroom pieces,shi

In [ ]:
print(len(main2sub))

57
